In [2]:
import os
import yt_dlp
import cv2
from ultralytics import YOLO
#from moviepy.editor import ImageSequenceClip

### 1. Yutube 영상 다운로드

In [3]:
# 다운로드할 YouTube URL
url = "https://www.youtube.com/shorts/DsXmREropi0"

# yt-dlp 설정
ydl_opts = {
    "format": "best",
    "outtmpl": "downloaded_video.%(ext)s",
}

# 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

[youtube] Extracting URL: https://www.youtube.com/shorts/DsXmREropi0
[youtube] DsXmREropi0: Downloading webpage
[youtube] DsXmREropi0: Downloading ios player API JSON
[youtube] DsXmREropi0: Downloading mweb player API JSON
[youtube] DsXmREropi0: Downloading player 89dfc5b3
[youtube] DsXmREropi0: Downloading m3u8 information
[info] DsXmREropi0: Downloading 1 format(s): 18
[download] Destination: downloaded_video.mp4
[download] 100% of    3.56MiB in 00:00:00 at 18.90MiB/s  


### 2. 동영상의 fps 정보

In [4]:
# 동영상 파일 경로
video_path = "downloaded_video.mp4"

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# FPS 가져오기
fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS:", fps)

# 동영상 파일 닫기
cap.release()

FPS: 25.0


### 3. 동영상을 프레임 단위로 분할

In [5]:
video_path = "downloaded_video.mp4"

# cv2를 이용해 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 프레임을 저장할 디렉토리 생성
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# 프레임 단위로 이미지로 저장
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # 이미지 파일로 프레임 저장
    frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

# 동영상 파일 닫기
cap.release()
print(f"총 {frame_count}개의 프레임이 저장되었습니다.")

총 1242개의 프레임이 저장되었습니다.


### 4. YOLOv8 추론

In [6]:
# Load a model
model = YOLO("yolov8n.pt")  # pretrained YOLO8n model

# "frames" 디렉토리 안에 있는 파일과 폴더 목록을 가져옴
frames = os.listdir("frames/")

# 원하는 디렉토리 경로 설정
directory_path = "results2"

# 디렉토리가 없으면 생성
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"디렉토리 '{directory_path}'가 생성되었어.")
else:
    print(f"디렉토리 '{directory_path}'가 이미 존재해.")

for frame in frames:
    # YOLOv8 모델을 사용하여 객체 감지 수행
    results = model("frames/" + frame,names=[0])  # , device="mps" (맥북)

    # Process results list
    for result in results:
        result.save(filename="results/" + frame)  # save to disk

100%|██████████| 6.25M/6.25M [00:00<00:00, 45.3MB/s]


디렉토리 'results'가 생성되었어.

image 1/1 /home/aaa/vision-ai-inference-practice/4. video-ai-inference/frames/frame_0416.jpg: 640x384 3 persons, 372.5ms
Speed: 36.6ms preprocess, 372.5ms inference, 19.3ms postprocess per image at shape (1, 3, 640, 384)



[W1127 17:15:31.265342567 NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


image 1/1 /home/aaa/vision-ai-inference-practice/4. video-ai-inference/frames/frame_0497.jpg: 640x384 2 persons, 1 sports ball, 234.2ms
Speed: 1.2ms preprocess, 234.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /home/aaa/vision-ai-inference-practice/4. video-ai-inference/frames/frame_1118.jpg: 640x384 5 persons, 233.1ms
Speed: 10.9ms preprocess, 233.1ms inference, 3.6ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /home/aaa/vision-ai-inference-practice/4. video-ai-inference/frames/frame_0271.jpg: 640x384 10 persons, 1 sports ball, 193.2ms
Speed: 4.8ms preprocess, 193.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /home/aaa/vision-ai-inference-practice/4. video-ai-inference/frames/frame_0754.jpg: 640x384 3 persons, 201.9ms
Speed: 2.2ms preprocess, 201.9ms inference, 3.5ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /home/aaa/vision-ai-inference-practice/4. video-ai-inference/frames/frame_1096

### 5. 프레임을 동영상으로 변환 - moviepy 라이브러리

In [ ]:
# 이미지 파일들이 저장된 디렉토리 경로
image_folder = "results"  # 이미지 파일들이 있는 폴더 경로
fps = int(fps)  # 초당 프레임 수 설정

# 이미지 파일 리스트 가져오기 (예: jpg, png 형식)
image_files = [
    os.path.join(image_folder, img)
    for img in sorted(os.listdir(image_folder))
    if img.endswith((".jpg", ".png"))
]

# 이미지 시퀀스를 이용해 클립 생성
clip = ImageSequenceClip(image_files, fps=fps)

# 동영상 파일로 저장
output_path = "output_video.mp4"
clip.write_videofile(output_path, codec="libx264")

print(f"동영상이 생성되었습니다: {output_path}")

In [7]:
image_folder = 'results'  # 이미지들이 저장된 폴더 경로
video_name = 'output_video.mp4'  # 저장할 비디오 파일 이름

# 이미지 파일 이름을 읽고 정렬 (순서대로 동영상을 만들기 위해)
images = [img for img in os.listdir(image_folder) if img.endswith(".png") or img.endswith(".jpg")]
images.sort()

# 첫 번째 이미지에서 프레임의 너비와 높이 가져오기
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

# 비디오 코덱과 파일명 설정, FPS 설정
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # mp4 코덱 설정
video = cv2.VideoWriter(video_name, fourcc, 30, (width, height))  # FPS를 30으로 설정

# 이미지를 순차적으로 불러와서 비디오에 추가
for image in images:
    frame = cv2.imread(os.path.join(image_folder, image))
    video.write(frame)

# 비디오 파일 저장 종료
video.release()
cv2.destroyAllWindows()